# __Использованные материалы__

* [LightFM Github](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/quickstart.html)
* [Google recommendation systems course](https://developers.google.com/machine-learning/recommendation)
* [Recommender system using Bayesian personalized ranking](https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9)
* [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/)
* [How to build a Movie Recommender System in Python using LightFm](https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b)
* [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=ratings_small.csv)

# __Краткое введение__

__LightFM__ - это реализация на Python'е ряда популярных алгоритмов рекомендаций, включая эффективную реализацию BPR и WARP. Он прост в использовании, быстр (благодаря многопоточности) и дает высококачественные результаты. <br>
Существуют две основные стратегии создания рекомендательных систем: 
* __Content-based Filtering__
* __Collaborative filtering__

На практике чаще всего они используются в совокупности.<br>
<em>Далее для удобства будет использоваться термин item, который подразумевает под собой сущности, рекомендуемые системой.</em>

### __Content-based Filtering__ 
Данный подход предполагает работу с метаданными пользователя, которые собираются различными способами:
* __explicit__ - пользователь заполняет анкеты для выявление предпочтений, к примеру оценивает какой-то item по дифференцированной шкале.<br>
* __implicit__ - все действия пользователя протоколируются для выявления предпочтений, к примеру переход по ссылками, информация о компьютере пользователя и тп.<br>

### __Collaborative filtering__ 
Данный подход использует группировку пользователей и item'ов по каким-то сходствам/критериям. Будет реализоваться следующая логика "Пользователям, которым понравился item $X$, также нравились item'ы $Y$". Похожесть как правило определяется следующими методами:<br>
* __Content-based__ - на основании характеристик item'ов и пользователей.<br>
* __Transaction-based__ - на основании того, входили ли item'ы в одну транзакцию, а пользователи совершали схожие действия.<br>

### Machine-learned ranking 
В __LightFM__ представлены два классических подхода MLR'а:
* __Bayesian Personalized Ranking (BLR)__ 
* __Weighted Approximate-Rank Pairwise (WARP)__ 

### Bayesian Personalized Ranking 
Основная идея заключается в выборке и попарном сравнение положительных и отрицательных item'ов. Алгоритм в упрощенном виде можно представить следующим образом:
1. Случайным образом возьмем пользователя $u$ и item $i$, который ранее был выбран пользователем, в таком случае item $i$ будет считаться <em>положительным.</em>
2. Случайным образом возьмем item $j$, который был выбран пользователем <em>реже</em>, чем $i$ (в том числе, который пользователь никогда не выбирал), в таком случае item $j$ будет считаться <em>отрицательным.</em>
3. Вычисляем оценку $p_{ui}$ и $p_{uj}$ пользователя $u$, а также положительного item'а $i$ и отрицательного item'а $j$ соответственно.
4. Находим разницу между положительными и отрицательными оценками, как $x_{uij} = p_{ui} - p_{uj}.$ 
5. Пропускаем эту разницу через сигмоид и используем ее для вычисления веса для обновления всех параметров модели с помощью градиентного шага(SGD).

### Weighted Approximate-Rank Pairwise
Концепция данного подхода схожа с BPR, за исключением случаев, когда происходит градиентный шаг:
* В BPR градиентный шаг происходит каждый раз с разницей в качестве веса.
* WARP совершает градиентный шаг только в случае неверного предсказания (т.е. оценка отрицательного item'а больше положительного). Если предсказание было верным, то продолжаем выбирать отрицательные item'ы, пока не получим неверный прогноз или не достигнем некоторого порогового значения.

Для этих целей WARP предоставляет два гиперпараметра:
1. __Margin__ - определяет насколько ошибочным должен быть прогноз для совершения градиентного шага. 
2. __Cutoff__ - определяет сколько раз мы готовы выбирать отрицательные примеры, пытаясь получить неверное предсказание, прежде чем откажемся и перейдем к следующему пользователю.

<em>Автор статьи [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/) утверждает, что на практике вероятнее всего WARP предпочтительнее для большинства рекомендательных систем, нежели BPR.</em>

# __Тестовый пример__
Попробуем реализовать простейшую рекомендательную систему на основе [датасета, предоставляемого LightFM'ом.](https://grouplens.org/datasets/movielens/100k/)
## __Установка зависимостей__
### __Виртуальное окружение__
Для его создания будет использоваться conda.
#### Установка conda для Windows:

In [ ]:
%%cmd
@"%SystemRoot%\System32\WindowsPowerShell\v1.0\powershell.exe" -NoProfile -InputFormat None -ExecutionPolicy Bypass -Command "[System.Net.ServicePointManager]::SecurityProtocol = 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))" && SET "PATH=%PATH%;%ALLUSERSPROFILE%\chocolatey\bin"
ECHO Y | choco install miniconda3 --params="'/AddToPath:1'"

#### Установка conda для Ubuntu:

In [ ]:
%%sh
sudo apt update --yes
sudo apt upgrade --yes

wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
bash ~/miniconda.sh -b -p ~/miniconda 
rm ~/miniconda.sh

export PATH=~/miniconda/bin:$PATH

#### Создаем и активируем виртуальное окружение c помощью команд <br>
```
conda create -n LightFM-env
conda activate LightFM-env
pip install --user ipykernel
python -m ipykernel install --user --name=LightFM-env
```
#### Затем добавляем новый кернел в нотбук
<em>По неведомым мне причинам подход к созданию виртуальной среды через conda в Windows упорно не хотел работать и активация виртуального окружения происходила только через cmd (Powershell отказывался работать). Полдня стараний зафиксить эту проблему не увенчались успехом, соответственно терминал был не доступен, поэтому дальше будет описан подход через venv.

In [1]:
import sys
from platform import python_version
if float(python_version()[:-2]) < 3.3: #Поскольку venv является стандартной библиотекой в Python начиная с версии 3.3.*
    print("Upgrade Python to use venv library features for correct further work")

In [2]:
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m venv LightFM-env

#### Windows

In [3]:
%%cmd
.\LightFM-env\Scripts\activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --name=LightFM-env

Microsoft Windows [Version 10.0.19042.1165]
(c) Microsoft Corporation. All rights reserved.

C:\A.Mindset\internship_ds\LightFM>.\LightFM-env\Scripts\activate

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>python -m pip install --upgrade pip
  Using cached pip-21.2.4-py3-none-any.whl (1.6 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>pip install ipykernel
  Using cached ipykernel-6.2.0-py3-none-any.whl (122 kB)
  Using cached tornado-6.1-cp37-cp37m-win_amd64.whl (422 kB)
  Using cached debugpy-1.4.1-cp37-cp37m-win_amd64.whl (4.4 MB)
  Using cached jupyter_client-7.0.0-py3-none-any.whl (122 kB)
  Using cached argcomplete-1.12.3-py2.py3-none-any.whl (38 kB)
  Using cached importlib_metadata-4.6.4-py3-none-any.whl (17 kB)
  Using cached matplotlib_inline-0.1.2-py3-none-any.whl (8.2 kB)
  Using cached ipython-7.26.0-py3-none-any.whl (786 

#### Затем перезапускаем нотбук через виртуальную среду и добавляем новый кернел

In [13]:
!jupyter kernelspec list

Available kernels:
  python3        c:\a.mindset\internship_ds\lightfm\lightfm-env\share\jupyter\kernels\python3
  lightfm-env    C:\ProgramData\jupyter\kernels\lightfm-env


#### Ubuntu

In [ ]:
%%sh
source LightFM-env/bin/activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --name=LightFM-env

In [ ]:
!jupyter kernelspec list

#### Затем перезапускаем нотбук через виртуальную среду и добавляем новый кернел

### Установка библиотек для тестового примера

In [2]:
!pip install lightfm
!pip install numpy
!pip install scipy

  Using cached lightfm-1.16.tar.gz (310 kB)
  Using cached numpy-1.21.2-cp37-cp37m-win_amd64.whl (14.0 MB)
  Using cached scipy-1.7.1-cp37-cp37m-win_amd64.whl (33.6 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached scikit_learn-0.24.2-cp37-cp37m-win_amd64.whl (6.8 MB)
  Using cached idna-3.2-py3-none-any.whl (59 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached charset_normalizer-2.0.4-py3-none-any.whl (36 kB)
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached threadpoolctl-2.2.0-py3-none-any.whl (12 kB)
Using legacy 'setup.py install' for lightfm, since package 'wheel' is not installed.
    Running setup.py install for lightfm: started
    Running setup.py install for lightfm: finished with status 'done'


In [19]:
#Импорт необходимых библиотек для тестового примера
import numpy as np
from lightfm.datasets import fetch_movielens #метод lightfm для извлечения данных фильма
from lightfm import LightFM

In [239]:
#Получаем данные фильма с минимальным рейтингом 4
data = fetch_movielens(min_rating = 4.0)

#Отобразим данные
print(repr(data['train']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>


In [5]:
#Создадим модель
model = LightFM(loss = 'warp')
#Тренировка
model.fit(data['train'], epochs=30, num_threads=2)

#Рекомендательная функция
def sample_recommendation(model, data, user_ids):
    #Число пользователей и фильмов в обучающем наборе
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
    	#Фильмы, которые уже понравились пользователям
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        #Предсказание фильмов, которые им понравится
        scores = model.predict(user_id, np.arange(n_items))
        #Сортирует результат по оценке
        top_items = data['item_labels'][np.argsort(-scores)]
        #Отображение результатов
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 451])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        Scream (1996)
        Air Force One (1997)
        Game, The (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        English Patient, The (1996)
        Contact (1997)
        Titanic (1997)
User 451
     Known positives:
        Twelve Monkeys (1995)
        Babe (1995)
        Mr. Holland's Opus (1995)
     Recommended:
        Raiders of the Lost Ark (1981)
        Amadeus (1984)
        Silence of the Lambs, The (1991)


# __Работа с kaggle датасетом__
Для этих целей возьмем датасет [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=keywords.csv)

In [6]:
!pip install kaggle

  Using cached kaggle-1.5.12.tar.gz (58 kB)
  Using cached tqdm-4.62.1-py2.py3-none-any.whl (76 kB)
  Using cached python_slugify-5.0.2-py2.py3-none-any.whl (6.7 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using legacy 'setup.py install' for kaggle, since package 'wheel' is not installed.
    Running setup.py install for kaggle: started
    Running setup.py install for kaggle: finished with status 'done'


Теперь нам необходимо создать API токен на kaggle по адресу `https://www.kaggle.com/<username>/account` и поместить его в папку .kaggle, расположение которой зависит от ОС:
* Для Windows - `C:\Users\<Windows-username>\.kaggle\kaggle.json`
* Для Linux систем - ```~/.kaggle/kaggle.json```

### Скачивание и распаковка датасета

In [11]:
from zipfile import ZipFile
import os
!mkdir LightFM-Dataset 
%cd .\LightFM-Dataset
!kaggle datasets download -d rounakbanik/the-movies-dataset
zip_file = ZipFile('the-movies-dataset.zip')
zip_file.extractall()
zip_file.close()
os.remove("the-movies-dataset.zip")
%cd ..

C:\A.Mindset\internship_ds\LightFM\LightFM-Dataset



  0%|          | 0.00/228M [00:00<?, ?B/s]
  0%|          | 1.00M/228M [00:00<00:33, 7.18MB/s]
  3%|3         | 7.00M/228M [00:00<00:08, 26.6MB/s]
  5%|5         | 12.0M/228M [00:00<00:06, 35.2MB/s]
 11%|#         | 24.0M/228M [00:00<00:03, 64.1MB/s]
 14%|#3        | 31.0M/228M [00:00<00:04, 43.7MB/s]
 16%|#6        | 37.0M/228M [00:00<00:05, 38.2MB/s]
 18%|#8        | 42.0M/228M [00:01<00:06, 32.0MB/s]
 20%|##        | 46.0M/228M [00:01<00:06, 29.0MB/s]
 22%|##1       | 50.0M/228M [00:01<00:06, 26.8MB/s]
 23%|##3       | 53.0M/228M [00:01<00:07, 25.6MB/s]
 25%|##4       | 56.0M/228M [00:01<00:07, 24.7MB/s]
 26%|##5       | 59.0M/228M [00:02<00:07, 25.1MB/s]
 27%|##7       | 62.0M/228M [00:02<00:06, 26.2MB/s]
 29%|##8       | 66.0M/228M [00:02<00:05, 28.7MB/s]
 30%|###       | 69.0M/228M [00:02<00:06, 26.9MB/s]
 32%|###1      | 72.0M/228M [00:02<00:06, 25.3MB/s]
 33%|###3      | 76.0M/228M [00:02<00:05, 27.7MB/s]
 35%|###4      | 79.0M/228M [00:02<00:05, 27.0MB/s]
 36%|###5      | 82.


C:\A.Mindset\internship_ds\LightFM


In [137]:
!pip install matplotlib
!pip install pandas
%matplotlib inline

  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached Jinja2-3.0.1-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.0.1-cp37-cp37m-win_amd64.whl (14 kB)
Using legacy 'setup.py install' for dash, since package 'wheel' is not installed.
Using legacy 'setup.py install' for dash-core-components, since package 'wheel' is not installed.
Using legacy 'setup.py install' for dash-html-components, since package 'wheel' is not installed.
Using legacy 'setup.py install' for dash-table, since package 'wheel' is not installed.
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
    Running setup.py install for future: started
    Running setup.py install for future: finished with status 'done'
    Running setup.py install for dash-table: started
    Running setup.py install for dash-table: finished with status 'done'
    Running setup.py install for dash-html-components: started
    Running setup.py install for dash-html-components: finished with status 'do

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import itertools
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix

C:\A.Mindset\internship_ds\LightFM\LightFM-env\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [2]:
movie_metadata = pd.read_csv('LightFM-Dataset/movies_metadata.csv', low_memory=False)[['id','original_title','overview','genres']].set_index('original_title').dropna()
movie_metadata

,id,overview,genres
original_title,,,
Toy Story,862,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
Jumanji,8844,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
Grumpier Old Men,15602,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
Waiting to Exhale,31357,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
Father of the Bride Part II,11862,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...
رگ خواب,439050,Rising and falling between a man and woman.,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
Siglo ng Pagluluwal,111109,An artist struggles to finish his work while a...,"[{'id': 18, 'name': 'Drama'}]"
Betrayal,67758,"When one of her hits goes wrong, a professiona...","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."


In [3]:
rating_dataset = pd.read_csv('LightFM-Dataset/ratings.csv', low_memory=False)[['userId','movieId','rating']].dropna()
rating_dataset

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [4]:
# Убираем фильмы и пользователей с малым количеством отзывов
filter_movies = (rating_dataset['movieId'].value_counts()>10000)
filter_movies = filter_movies[filter_movies].index.tolist()

filter_users = (rating_dataset['userId'].value_counts()>200)
filter_users = filter_users[filter_users].index.tolist()

rating_dataset_filtered = rating_dataset[(rating_dataset['movieId'].isin(filter_movies)) & (rating_dataset['userId'].isin(filter_users))]
del filter_movies, filter_users
print('Shape User-Ratings unfiltered:\t{}'.format(rating_dataset.shape))
print('Shape User-Ratings filtered:\t{}'.format(rating_dataset_filtered.shape))

Shape User-Ratings unfiltered:	(26024289, 3)
Shape User-Ratings filtered:	(6655935, 3)


In [5]:
rating_dataset_filtered.head(10)

,userId,movieId,rating
414,11,32,3.5
415,11,34,4.0
416,11,47,3.5
417,11,110,3.5
418,11,165,3.5
419,11,231,2.5
420,11,260,3.0
421,11,296,4.0
422,11,318,4.5
423,11,344,3.0


In [6]:
rating_dataset_filtered_shuffled = rating_dataset_filtered.sample(frac=1).reset_index(drop=True)
rating_dataset_filtered_shuffled.head(10)

,userId,movieId,rating
0,16087,150,1.5
1,57772,185,4.0
2,205780,1250,5.0
3,29789,780,4.0
4,140746,1247,3.5
5,226886,3481,3.5
6,99350,1923,4.0
7,143399,95,2.5
8,146783,5991,5.0
9,38495,44191,4.5


In [7]:
n = 1000000
rating_dataset_train = rating_dataset_filtered_shuffled[:-n]
rating_dataset_test = rating_dataset_filtered_shuffled[-n:]

In [8]:
rating_dataset_train

,userId,movieId,rating
0,16087,150,1.5
1,57772,185,4.0
2,205780,1250,5.0
3,29789,780,4.0
4,140746,1247,3.5
...,...,...,...
5655930,256652,235,3.0
5655931,201405,4014,4.0
5655932,164071,924,2.5
5655933,159676,2700,3.5


In [9]:
#Создадим User-Movie-matrix
user_movie_matrix = rating_dataset_train.pivot_table(index='userId', columns='movieId', values='rating')
print('Shape User-Movie-Matrix:\t{}'.format(user_movie_matrix.shape))
user_movie_matrix.sample(3)

Shape User-Movie-Matrix:	(32811, 636)


movieId,1,2,3,5,6,7,10,11,16,17,...,99114,106782,109487,112556,112852,116797,122882,122886,134130,134853
userId,,,,,,,,,,,,,,,,,,,,,
1602,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170372,3.0,4.0,NaN,NaN,4.5,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,2.0,NaN,NaN,3.0


In [10]:
#Создадим маппинг для пользователей и фильмов
user_id_mapping = {id:i for i, id in enumerate(rating_dataset_filtered['userId'].unique())}
movie_id_mapping = {id:i for i, id in enumerate(rating_dataset_filtered['movieId'].unique())}
#Применим его к обучающему и тренировочному набору
train_user_data = rating_dataset_train['userId'].map(user_id_mapping)
train_movie_data = rating_dataset_train['movieId'].map(movie_id_mapping)

test_user_data = rating_dataset_test['userId'].map(user_id_mapping)
test_movie_data = rating_dataset_test['movieId'].map(movie_id_mapping)

In [11]:
#Создадим разреженную матрицу рейтинга
shape = (len(user_id_mapping), len(movie_id_mapping))
train_matrix = coo_matrix((rating_dataset_train['rating'].values, (train_user_data.astype(int), train_movie_data.astype(int))), shape=shape)
test_matrix = coo_matrix((rating_dataset_test['rating'].values, (test_user_data.astype(int), test_movie_data.astype(int))), shape=shape)

In [12]:
#Создадим модель LightFM и обучим ем
model = LightFM(loss='warp')
%timeit model.fit(train_matrix, epochs=30, num_threads=2)

9min 4s ± 12.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
k = 20
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, train_matrix, k=k).mean()))
print('Test precision at k={}:\t\t{:.4f}'.format(k, precision_at_k(model, test_matrix, k=k).mean()))

Train precision at k=20:	0.7917
Test precision at k=20:		0.1182


In [50]:
#Старая версия модели с урезанным датасетом показывала подобные результаты
k = 20
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, train_matrix, k=k).mean()))
print('Test precision at k={}:\t\t{:.4f}'.format(k, precision_at_k(model, test_matrix, k=k).mean()))

Train precision at k=20:	0.9580
Test precision at k=20:		0.0138
